In [15]:
import pandas as pd
import numpy as np
from numpy import linalg
import os, sys

from utility.praser import deleteNAN
from matplotlib import pyplot as plt
from scipy import stats

In [16]:
# meta-data
data_types = ['cardboard', 'foam', 'metal', 'plastic', 'wooden-cabinet', 'wooden-shelf']
meas_numbers = 6
type_encoding = {
    'nothing':0,
    'cardboard':1, 
    'foam':2, 
    'metal':3, 
    'plastic':4, 
    'wooden-cabinet':5, 
    'wooden-shelf':6
}

In [26]:
# collecting all relevant files
df_list = []
pos_list = []
for data_type in data_types:
    for num in range(1,meas_numbers):
        # collect csv
        path = f"../../my_data/identification-dataset/nlos/anTag/{data_type}/data{num}/{data_type}-anTag-data{num}_data.csv"
        df = pd.read_csv(path)
        df['material'] = data_type
        # add material encoding
        df['encoded_material'] = type_encoding[df['material'].unique()[0]]
        # save df
        df_list.append(df)

        # collect pos
        folder = f"../../my_data/identification-dataset/nlos/anTag/{data_type}/data{num}/"
        # search for txt files in the folder
        for file in os.listdir(folder):
            if file.endswith(".txt"):
                pose_txt = os.path.join(folder, file)
        
        f = open(pose_txt,"r")
        pos=[];    quat=[]       # position and quaternion
        for line in f:
            x = line.split(",")
            if len(x) == 4:
                arr_x = [float(x[1]), float(x[2]), float(x[3])]
                pos.append(arr_x)
            if len(x) == 5:
                q_x = [float(x[1]), float(x[2]), float(x[3]), float(x[4])]
                quat.append(q_x)

        # calculate values
        pos = np.array(pos)     # [an1_p, an2_p, tag_p]
        quat = np.array(quat)   # [an1_quat, an2_quat, tag_quat]
        an1_p = pos[0,:]; 
        an2_p = pos[1,:]; 
        tag_p = pos[2,:]
        obs = np.vstack((pos[3,:], pos[4,:], pos[5,:], pos[6,:]))
        # obs_up = pos[3:,:]
        # obs_bt = np.copy(obs_up);  obs_bt[:,2] = 0 
        # obstacle = np.concatenate((obs_up, obs_bt), axis=0)

        # add to list
        # pos_list.append([pos, quat, an1_p, an2_p, tag_p, obs_up, obs_bt, obstacle])
        pos_list.append([pos, quat, an1_p, an2_p, tag_p, obs])


In [ ]:
# splitting and relabeling the data
splitted_df_list = []
for df, pos in zip(df_list, pos_list):

    # extract data
    tdoa12 = deleteNAN(np.array(df['tdoa12']))
    tdoa21 = deleteNAN(np.array(df['tdoa21']))


    tag_p = pos[4]
    an1_p = pos[2]
    an2_p = pos[3]

     # compute ground truth (gt)
     